In [ ]:
%cd '/content/drive/MyDrive/Movie Rating Prediction/Movie Reviews Weighted Average'

/content/drive/MyDrive/Movie Rating Prediction/Movie Reviews Weighted Average


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools

pd.options.display.max_colwidth=500

In [ ]:
def getSoup(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text , 'html.parser')
  return soup

In [ ]:
# API call to select:
## feature films
## which are rated atleast 4.0
## having atleast 50,000 votes
## in the Thriller genre
## sorted by user rating
## limit to 250 movies

# urls = ['https://www.imdb.com/search/title/?title_type=feature&user_rating=4.0,10.0&num_votes=50000,&genres=thriller&view=simple&sort=user_rating,desc&count=250' + str()]
# url_bot = 'https://www.imdb.com/search/title/?title_type=feature&user_rating=4.0,10.0&num_votes=50000,&genres=thriller&view=simple&sort=user_rating,asc&count=250'

urls = ['https://www.imdb.com/search/title/?title_type=feature&user_rating=4.0,10.0&num_votes=50000,&genres=thriller&view=simple&sort=user_rating,desc&start=' + str(i) for i in range(1200,1250,25)]

# get the soup object for main api url
# movies_soup_top = getSoup(url_top)
# movies_soup_bot = getSoup(url_bot)

movies_soups = [getSoup(url) for url in urls]

In [ ]:
movies_tags = []
for movies_soup_top in movies_soups:
    movie_tags_top = movies_soup_top.find_all('a' , attrs = {'class' : None})

    movie_tags_top = [tags.attrs['href'] for tags in movie_tags_top if tags.attrs['href'].startswith('/title') & tags.attrs['href'].endswith('/')  ]

    movie_tags_top = list(dict.fromkeys(movie_tags_top))

    movies_tags.append(movie_tags_top)


    print("There are a total of " + str(len(movie_tags_top)) + " movie titles")
    # print("Displaying 10 titles")
    # movie_tags_top[:10]


There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
There are a total of 50 movie titles
T

In [ ]:

movie_tags = []
for sublist in movies_tags:
    for item in sublist:
        movie_tags.append(item)
len(movie_tags)


2441

In [ ]:
# movie_tags_bot = movies_soup_bot.find_all('a' , attrs = {'class' : None})

# movie_tags_bot = [tags.attrs['href'] for tags in movie_tags_bot if tags.attrs['href'].startswith('/title') & tags.attrs['href'].endswith('/')  ]

# movie_tags_bot = list(dict.fromkeys(movie_tags_bot))

# print("There are a total of " + str(len(movie_tags_bot)) + " movie titles")
# print("Displaying 10 titles")
# movie_tags_bot[:10]


In [ ]:
# movie_tags = movie_tags_top + movie_tags_bot
print("There are a total of " + str(len(movie_tags)) + " movie titles")
print("Displaying 10 titles")
movie_tags[:10]


There are a total of 2441 movie titles
Displaying 10 titles


['/title/tt0468569/',
 '/title/tt1375666/',
 '/title/tt6751668/',
 '/title/tt0114369/',
 '/title/tt0102926/',
 '/title/tt0482571/',
 '/title/tt0407887/',
 '/title/tt0114814/',
 '/title/tt0110413/',
 '/title/tt0054215/']

In [ ]:
overall_ratings = []
for movies_soup_top in movies_soups:
    overall_rating_tags_top = movies_soup_top.find_all('strong' , attrs = {'class' : None})
    overall_rating_tags_top = [tags.get_text().strip() for tags in overall_rating_tags_top[2:] ]

    # overall_rating_tags_bot = movies_soup_bot.find_all('strong' , attrs = {'class' : None})
    # overall_rating_tags_bot = [tags.get_text().strip() for tags in overall_rating_tags_bot[2:] ]

    overall_ratings.append(overall_rating_tags_top)

len(overall_ratings)
# print(movie_tags[2:].get_text().strip())

# movie_tags = list(dict.fromkeys(movie_tags))

# print("There are a total of " + str(len(movie_tags)) + " movie titles")
# print("Displaying 10 titles")
# movie_tags[:10]


50

In [ ]:
overall_ratings_temp = overall_ratings.copy()
overall_ratings = []
for sublist in overall_ratings_temp:
    for item in sublist:
        overall_ratings.append(item)
len(overall_ratings)

2441

In [ ]:
base_url = 'https://www.imdb.com/'
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]

print("There are a total of " + str(len(movie_links)) + " movies with user reviews")
print("Displaying 10 user reviews links")
movie_links[-10:]

There are a total of 2441 movies with user reviews
Displaying 10 user reviews links


['https://www.imdb.com//title/tt2322441/reviews',
 'https://www.imdb.com//title/tt0111301/reviews',
 'https://www.imdb.com//title/tt4477536/reviews',
 'https://www.imdb.com//title/tt0329774/reviews',
 'https://www.imdb.com//title/tt1564585/reviews',
 'https://www.imdb.com//title/tt1241317/reviews',
 'https://www.imdb.com//title/tt0211443/reviews',
 'https://www.imdb.com//title/tt1071875/reviews',
 'https://www.imdb.com//title/tt2322441/reviews',
 'https://www.imdb.com//title/tt0111301/reviews']

In [ ]:
movie_soups = [getSoup(link) for link in movie_links]


In [ ]:
def getReviews(soup):
    '''Function returns a negative and positive review for each movie.'''
    
    
    # get the review tags
    user_review_list = soup.find_all('a', attrs={'class' : 'title'})

    

    user_review_links = ["https://www.imdb.com" + user_review_tag['href'] for user_review_tag in user_review_list]

    
    return user_review_links


In [ ]:
movie_review_links  = [getReviews(movie_soup) for movie_soup in movie_soups]


In [ ]:
movie_review_links[0][0]

'https://www.imdb.com/review/rw6467314/'

In [ ]:

def getReviewDetails(review_url):
    soup = getSoup(review_url)

    tag = soup.find('div', attrs={'class': 'text show-more__control'})

    text = tag.getText()

    rating = soup.find_all('span', attrs={'class': None})[1].getText()

    tag = soup.find('h1')
    title = list(tag.children)[1].getText()

    tag = soup.find('div', attrs={'class': 'actions text-muted'})
    num_helpful = tag.get_text().split()[0]
    total_number = tag.get_text().split()[3]

    return text , title , rating , num_helpful , total_number 




In [ ]:
# def getReviewText(review_url):
#     '''Returns the user review text given the review url.'''
    
#     # get the review_url's soup
#     soup = getSoup(review_url)
    
#     # find div tags with class text show-more__control
#     tag = soup.find('div', attrs={'class': 'text show-more__control'})
    
#     return tag.getText()

# def getReviewRating(review_url):
#     '''Returns the user review rating given the review url.'''

#     # get the review_url's soup
#     soup = getSoup(review_url)
    
#     # find div tags with class text show-more__control
#     rating = soup.find_all('span', attrs={'class': None})
    
#     return rating[1].get_text()




# def getMovieTitle(review_url):
#     '''Returns the movie title from the review url.'''
    
#     # get the review_url's soup
#     soup = getSoup(review_url)
    
#     # find h1 tag
#     tag = soup.find('h1')
    
#     return list(tag.children)[1].getText()


In [ ]:
count = 0
review_texts = []
movie_titles  = []
reviewer_ratings = []
helpful_nums = []
total_nums = []
overall_movie_ratings = []
for user_review_link_list in movie_review_links[1270:2070]:
  for user_review_link in user_review_link_list:
    
    review_text , movie_title , reviewer_rating , helpful_num , total_num = getReviewDetails(user_review_link)
    if '\n' not in reviewer_rating:
      review_texts.append(review_text)
      reviewer_ratings.append(reviewer_rating)
      movie_titles.append(movie_title)
      helpful_nums.append(helpful_num)
      total_nums.append(total_num)
      overall_movie_ratings.append(overall_ratings[count])

  count += 1
  if count % 10 == 0:
    print(str(count))
        


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800


In [ ]:
print(len(review_texts))
print(len(reviewer_ratings))
print(len(movie_titles))
print(len(helpful_nums))
print(len(total_nums))
print(len(overall_movie_ratings))

18289
18289
18289
18289
18289
18289


In [ ]:
df = pd.DataFrame({'movie title': movie_titles , 'overall review' : overall_movie_ratings , 'user_review': review_texts, 'user_rating': reviewer_ratings , 'Helpful Count' : helpful_nums , 'Total Count' : total_nums})

In [ ]:
df.head()

,movie title,overall review,user_review,user_rating,Helpful Count,Total Count
0,Suspiria,9,"Most people call this movie out as ""bland"" or ""pretentious"" or get upset this doesn't have any ""classic"" horror. they are dead wrong. if you are a fan of the original, you will appreciate this piece for its sheer contrast but similarity to its predecessor. It is beautifully dark, and a wonderful, nightmareish horror flick. This movie is unconventional and different with its' dynamic and cinematography. Dakota Johnson and Tilda Swinton shine with this movie.Prepare for an assault on the sense...",10,30,39
1,Suspiria,9,"One reviewer said that if you liked 'The Shining"" you would love this film - I liked The Shining but found Suspiria to be tedious, pretentious, overlong and lacking a strong narrative. To make an effective 152 minute horror film would require a strong screenplay and a great director/editor and Suspiria is sadly lacking in these areas. Nearly every recent film could do with removing about thirty minutes and Suspiria does not have the content for more than 90 minutes but here we have film-make...",1,407,709
2,Suspiria,9,"As a fan of Dario Argento's original, I was cautiously hopeful about this remake, but ended up feeling really let down. This remake doesn't have the pervasive sense of inevitable doom of the original, nor does it deliver on the scares. While it is punctuated with some good performances by some of the cast, it lacks the visual punch of the original, and replaces the original's sense of dread with occasional moments of uninspired gore.The ending is likewise dissatisfying, giving us a long sequ...",1,323,563
3,Suspiria,9,"I can understand why this film has been so polarizing. With that said, know that I'm clinging to the side that thinks this movie is an absolute masterpiece. I'm a huge fan of Argento's legendary 1977 film, but aside from title, character names, and general plot, this film is entirely separate from that technicolor nightmare fairy tale. Guadagnino has created a dour, political, and animalistic horror epic that's closer in style to a Fassbinder film or Kubrick's The Shining than anything in th...",10,625,"1,118"
4,Suspiria,9,"Suspiria, as I'm sure anyone reading this is aware, is a remake of the 1977 Dario Argento film of the same name. This time around, it's directed by Luca Guadagnino (Call me by Your Name) and David Kajganich (who worked by Guadagnino on A Bigger Splash and created the AMC limited television show ""The Terror""), and expands upon the folklore of Agento's ""Three Mothers"" trilogy. 2018's Suspiria is less a remake and more of an expansion on the world of the Three Mothers, even boasting a post-cred...",4,561,"1,008"


In [ ]:
df.shape

(18289, 6)

In [ ]:
# save the dataframe to a csv file.
df.to_csv('userReviews14.csv', index=False)

# pickle the dataframe
df.to_pickle('userReviews14.pkl')

# to validate
#temp = pd.read_csv('userReviews1.csv')
temp = pd.read_pickle('userReviews14.pkl')

In [ ]:
%cd '/content/drive/MyDrive/Movie Rating Prediction/Movie Reviews Weighted Average'

/content/drive/MyDrive/Movie Rating Prediction/Movie Reviews Weighted Average


In [ ]:
# # save the dataframe to a csv file.
# df.to_csv('userReviews.csv', index=False)

# # pickle the dataframe
# df.to_pickle('userReviews.pkl')

# to validate
#temp = pd.read_csv('userReviews.csv')
temp1 = pd.read_pickle('Scraped data/userReviews.pkl')

In [ ]:
temp1.shape

(11321, 6)

In [ ]:
temp1['user_rating'].value_counts()

10    2693
8     1799
9     1618
1     1324
7     1180
6      787
5      541
3      501
4      456
2      422
Name: user_rating, dtype: int64